# Word Embedding Models: word2vec #

*Lauren F. Klein wrote version 1.0 of this notebook based on the [Advanced Topics in Word Vectors workshop](https://dh2018.adho.org/en/machine-reading-part-ii-advanced-topics-in-word-vectors/) at DH 2018 as well as tutorials by [Radim Rehurek](https://rare-technologies.com/word2vec-tutorial/) and [Chris McCormick](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/)** 

### What is a word vector? ###

A word vector or embedding is a **numerical representation** of a word within a corpus based on co-occurence with other words. Linguists have found that much of the meaning of a word can be derived from looking at its surrounding context. Today, we will explore word2vec, which is one popular approach to representing words in a numerical format. Conveniently, word2vec is implemented in gensim, which we used last class for topic modeling, so we can use that library again.

### Everything gets a vector! ###

![oprah vector](http://lklein.com/wp-content/uploads/2021/10/oprah-everyone-3.png)

We've actually already been exploring vectors of words: scikit-learn's `CountVectorizer()`, which we used to create the docuemnt-term matrix for our tf-idf calculations. But that looked at individual words in relation to a corpus. 

Today, we're going to look at words in relation to other words.

To start, let's take a look at [an example of word2vec in action](http://benschmidt.org/profGender/).

### What is Word2Vec ###

So, what is this word2vec that Schmidt used to make the website?

Word2Vec is a neural-network or deep learning based approach of generating word vectors. There are many resources out there that will go into the heavy details of deep learning in general or deep learning for NLP such as Yoav Goldberg's Neural Network Methods in Natural Language Processing (Morgan & Claypool Publishers, 2017). Today, you'll get a high level overview -- just enough for you to understand what w2v is doing. 


### Training the neural network ### 

Word2Vec uses a trick you may have seen (or heard about) elsewhere in machine learning. We’re going to train a simple neural network with a single hidden layer to perform a certain task, but then we’re not actually going to use that neural network for the task we trained it on! 

Instead, the goal is actually just to learn the weights of the hidden layer–we’ll see that these weights are actually the “word vectors” that we’re trying to learn.

**Note:** Neural networks are basically a bunch of weights in the form of matrices. If you have lots of these matrices multiplies in a row, you get layers that make your network 'deep' - hence the name deep learning. 

Usually if your network has one hidden (or projection) it's called a 'deep' network. The 'neurons' are just functions that transform your data non-linearly. Each layer of the network will tranform your data so your weights become more sophisticated (and meaningful?) with each layer.

![neural network](http://lklein.com/wp-content/uploads/2019/10/mikolov.png)




### The Fake Task ###

So now we need to talk about this “fake” task that we’re going to build the neural network to perform, and then we’ll come back later to how this indirectly gives us those word vectors that we are really after.

We’re going to train the neural network to do the following. Given a specific word in the middle of a sentence (the input word), look at the words nearby and pick one at random. The network is going to tell us the probability for every word in our vocabulary of being the “nearby word” that we chose.

When I say "nearby", there is actually a "window size" parameter to the algorithm. A typical window size might be 5, meaning 5 words behind and 5 words ahead (10 in total).

The output probabilities are going to relate to how likely it is find each vocabulary word nearby our input word. For example, if you gave the trained network the input word “Soviet”, the output probabilities are going to be much higher for words like “Union” and “Russia” than for unrelated words like “watermelon” and “kangaroo”.

We’ll train the neural network to do this by feeding it word pairs found in our training documents. The below example shows some of the training samples (word pairs) we would take from the sentence “The quick brown fox jumps over the lazy dog.” I’ve used a small window size of 2 just for the example. The word highlighted in blue is the input word.

![skip-grams](http://mccormickml.com/assets/word2vec/training_data.png)

The network is going to learn the statistics from the number of times each pairing shows up. So, for example, the network is probably going to get many more training samples of (“Soviet”, “Union”) than it is of (“Soviet”, “Sasquatch”). When the training is finished, if you give it the word “Soviet” as input, then it will output a much higher probability for “Union” or “Russia” than it will for “Sasquatch”.

I've described something called the skim-gram methods of generating word vectors. THere's also another popular method called CBOW (continuous bag of words). The main difference is that while skip gram learns vectors by predicting the context words that come before and after our given word $w$, CBOW predicts the center word $w$ given context words.


## Let's try it out!

## Import gensim, nltk tokenizers, glob, and Path

In [ ]:
import gensim # remember this! 
from nltk.tokenize import sent_tokenize
from nltk.tokenize.treebank import TreebankWordTokenizer
import nltk
nltk.download('punkt')
import glob
from pathlib import Path

## Read in our corpus

In [ ]:
# next, we need to create our corpus.
# we will continue to use the Emory Wheel corpus

import os

base_dir = "../corpora/emory-wheel/articles/" 

all_docs = [] # our list which will store the text of each doc; empty for now

docs = os.listdir(base_dir) # get a list of all the files in the directory

for doc in docs: # iterate through the docs
    if not doc.startswith('.'): # get only the .txt files
        with open(base_dir + doc, "r", encoding="utf-8") as file: # force unicode conversion to keep PCs happy
            text = file.read() # read in the file as a single text string
            all_docs.append(text) # append it to the all_docs list

# lastly, just check the length of all_docs to see if it's 147
len(all_docs)

## Preprocessing

gensim takes text at the level of `sentences` as its input. 

So let's define a function that a takes a list of texts (e.g. our all_docs list) and converts it into sentences for gensim word2vec to use. The function will lower-case text and tokenize by sentence and word.

In [ ]:
# need our handy nltk tokenizer 
tokenizer = TreebankWordTokenizer()

# and we'll get titles
directory = "../corpora/emory-wheel/articles/"
files = glob.glob(f"{directory}/*.txt")
wheel_titles = [Path(file).stem for file in files]

# and the function
def make_sentences(list_txt):
    all_txt = []
    counter = 0
    for txt in list_txt:
        lower_txt = txt.lower()
        sentences = sent_tokenize(lower_txt)
        sentences = [tokenizer.tokenize(sent) for sent in sentences]
        all_txt += sentences
        print(wheel_titles[counter]) # let's print the title of the article
        print("Sentences: " + str(len(sentences)))  # let's check how many sentences there are per article
        counter += 1
    return all_txt

In [ ]:
# now let's run it
sentences = make_sentences(all_docs)

## Train model

Now that we have our corpus ready for gensim, we can train the model. To do so, we call the function `gensim.models.Word2Vec()`. This function has a couple dozen parameters, some of which are more important than others.

Here are a few major ones. Only two are MANDATORY: these are marked with an asterisk:

1. `sentences*`: This is where you provide your data. It must be in a format of iterable of iterables.
2. `sg`: Your choice of training algorithm. There are two standard ways of training W2V vectors -- 'skipgram' and 'CBOW'. If you enter 1 here the skip-gram is applied; otherwise, the default is CBOW.
3. `size*`: This is the length of your resulting word vectors. If you have a large corpus (>few billion tokens) you can go up to 100-300 dimensions. Generally word vectors with more dimensions give better results.
4. `window`: This is the window of context words you are training on. In other words, how many words come before and after your given word. A good number is 4 here but this can vary depending on what you are interested in. For instance, if you are more interested in embeddings that embody semantic meaning, smaller window sizes work better. 
5. `alpha`: The learning rate of your model. If you are interested in machine learning experimentation with your vectors you may experiment with this parameter.
6. `seed` (int): This is the random seed for your random initialization. All deep learning models initialize the weights with random floats before training. This is a useful field if you want to replicate your experiments because giving this a seed will initialize 'randomly' deterministically.
7. `min_count`: This is the minimum frequency threshold. If a given word appears with lower frequency than provided it will be ignored. This is here because words with very low frequency are hard to train.
8. `iter`: This is the number of iterations(entire run) over the corpus, also known as epochs. Usually anything between 1-10 is ok. The trade offs are that if you have higher iterations, it will take longer to train and the model may overfit on your dataset. However, longer training will allow your vectors to perform better on tasks relevant to your dataset.

Most of these settings will not concern us. As you'll see below, we are only going to use four arguments.

In [ ]:
# let's train our model!
wheel_model = gensim.models.Word2Vec(
    sentences,
    min_count=2, # default is 5; this trims the corpus for words only used once; 
    vector_size=200, # size of NN layers; default is 100; higher for larger corpora
    workers=5) # parallel processing; needs Cython

Hooray! We have a trained word2vec model: `wheel_model`!

## Save model — and load it

It's often useful to save your trained model to disk so that you can reload it as needed. 

In [ ]:
wheel_model.save('wheel_model')

And you can load a model in the same way (remember this from our topic model)

In [ ]:
old_model = gensim.models.Word2Vec.load('wheel_model') 

## What's in the model?

The method `wv.vocab` allows us to see the words in our model

In [ ]:
# take a quick look at the vocab
wheel_model.wv.vocab

## Let's play!

### Similarity

word2vec can tell us which words, according to its model, are most similar to any other. We call `model.wv.most_similar("word", topn=number of similar words)`. Let's try `class`

In [ ]:
# testing some basic functions

# basic similarity
wheel_model.wv.most_similar("class", topn=10)

Let's think about the results. `class` and `course` make intuitive sense: they have similar meanings to each other. But what's `study`? It's something you do for a class.

Our model's parallel invites our critical analysis: rather than take these results as *true*, we need to reflect, in part through turning to articles themselves and close reading them, on how the model arrives at its results, and what they might imply.

**Remember**: everything the model knows it knows from our corpus. What we're learning are assumptions *immanent* to the corpus.

## Exercise 1:

**Copy the code above and test out some words until you find one that has some interesting (or problematic) similar words.**

In [ ]:
# your answer here 

## Similarity between two words

We can choose specific words to compare with `model.wv.similarity(w1="word_one",w2="word_two")`

In [ ]:
# similarity b/t two words

print(wheel_model.wv.similarity(w1="professor",w2="teacher"))
print(wheel_model.wv.similarity(w1="professor",w2="friend"))

As expected, the model believes that your professor is more similar to your teacher than your friend. 🙁

### Analogy

We can also play with analogy tasks. The commonly seen task is:

'Man is to King as Woman is to ____?'

The general structure is:
`A is to A\*  as  B is to B\*`
                         
gensim provides two different ways of implementing this task. You may be familiar with the the additive version also called the 3CosAdd method:

$$\underset{b*\in V}{\textrm{arg max}} (cos(b*,b) - cos(b*,a) + cos(b*,a*))$$

This reflects the abstraction of Woman - Man + King. In this maximization, we are searching which word vector will allow us to produce the highest value in this equation.

We can implement this method with a built-in function. Positive here refers to words that give the positive contribution to similarity (nominator), and negative refers to words that contribute negatively (denominatory). Here's the additive method.

In [ ]:
# analogies
# format is: "man is to professor as woman is to ???"

result = wheel_model.wv.most_similar(positive=['woman', 'professor'], negative=['man'])
print("{}: {:.4f}".format(*result[0])) # this prints the top result

Man is to woman as professor is to...associate!? Ah, the bias of the model....

## Exercise 2:

**Copy the code above and test out some analogies until you find one that gets you some interesting (or problematic) results.**

In [ ]:
# your code here 

## There's so much more!

gensim has quite a few built-in tools, and it's worth taking some time to see what's available. Check the documentation here: [https://radimrehurek.com/gensim/models/keyedvectors.html](https://radimrehurek.com/gensim/models/keyedvectors.html)


## BONUS: Visualization!

Find below some code you can use to make visualizations from your word2vec model. We can't visualize all the many dimensions in our model, so we need to reduce them to two dimensions for our meager human brains. We do that with something called principal component analysis (PCA). 

Don't worry about the details for now. This is just a fun way to take a look at the output of our model. 

**Remember**: Our visualization reduces MANY dimensions to two, so a lot of information is lost.

In [ ]:
### Let's do some visualization ###

import numpy as np

# Get the interactive Tools for Matplotlib
%matplotlib notebook
import matplotlib.pyplot as plt
plt.style.use('ggplot')

from sklearn.decomposition import PCA

from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors

In [ ]:
def display_pca_scatterplot(model, words=None, sample=0):
    if words == None:
        if sample > 0:
            words = np.random.choice(list(model.wv.vocab.keys()), sample)
        else:
            words = [ word for word in model.wv.vocab ]
        
    word_vectors = np.array([model[w] for w in words])

    twodim = PCA().fit_transform(word_vectors)[:,:2]
    
    plt.figure(figsize=(6,6))
    plt.scatter(twodim[:,0], twodim[:,1], edgecolors='k', c='r')
    for word, (x,y) in zip(words, twodim):
        plt.text(x+0.05, y+0.05, word)

In [ ]:
display_pca_scatterplot(wheel_model, ['exam','stress','finals','rest','sleep'])

# display_pca_scatterplot(ccp_model, sample=20)

## Exercise 3:

Play around with the words in the cell above to plot some words that you think might be similar or different from each other. What do you think the plot shows you about the words? Did they confirm or contradict what you though they would show?

In [ ]:
# type your answer here 